# PROJET SEATTLE ENERGY BENCHMARKING
## Notebook 06 : pipeline de modélisation et premières expérimentations
---

### Identité du document
* **Statut :** Phase 1 (exploration & prototypage)
* **Dernière mise à jour :** 10/01/2026
* **Dépendances notebooks**: Notebooks 0 à 7,feature engineering

### Description
Ce notebook constitue la première étape de la construction des modèles prédictifs. Il s’appuie sur les données nettoyées et enrichies afin de tester plusieurs approches de modélisation, comparer leurs performances et établir une base de référence. L’objectif est de documenter un pipeline reproductible et d’identifier les modèles les plus prometteurs.


### Objectifs principaux
1. Charger le dataset préparé depuis `processed/`.  
2. Mettre en place le split train/test.  
3. Entraîner les modèles baseline :  
   - Régression linéaire.  
   - Régression Ridge/Lasso.  
   - Random Forest.  
4. Évaluer les performances avec RMSE, MAE, R².  
5. Intégrer MLflow pour tracer les runs (paramètres, métriques, artefacts).  
6. Documenter les résultats et générer un rapport synthétique.

---
### Dépendances critiques
* `src.feature_engineering` : pipeline de features.  
* `src.utils` : fonctions de split et métriques.  
* `sklearn` : librairie de modélisation.  
* `mlflow` : suivi des expériences.

### LIVRABLES
1. Modèles baseline entraînés et sauvegardés (`outputs/models/`).  
2. Résultats des métriques dans MLflow UI.  
3. Tableaux comparatifs des performances (`reports/model_baseline.md`).  
4. Visualisations des résidus et des distributions d’erreurs.  
5. Notebook documenté et reproductible via des scripts

---

# 📚 Table des matières

- [Section 0 : Importation des packages](#section-0)  
- [Section 1 : Chargement des données feature engineering](#section-1)  
- [Section 2 : Préprocessing et split train/test](#section-2)  
- [Section 3 : Entraînement des modèles basiques](#section-3)  
- [Section 4 : Évaluation des performances](#section-4)  
- [Section 5 : Intégration MLflow](#section-5)  
- [Section 6 : Documentation et synthèse](#section-6)  

> Note : la table des matières est indicative. Utilisez la navigation intégrée de votre éditeur (ex. outline VSCode) pour accéder rapidement aux sections.
---

<a id="section-0"></a>
# Section 0 : Importation des packages

In [1]:
import logging
import pandas as pd
from pathlib import Path


# Import des fonctions utilitaires 

import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
SRC_PATH = PROJECT_ROOT / "src"

if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))


from data.load_data import load_data_raw
from utils.config_loader import load_config, create_directories
from utils.eda_logger import setup_eda_logger

import seaborn as sns
import matplotlib.pyplot as plt

#pipeline
from utils.config_loader import load_config
from data.load_data import load_data_raw
from data.clean_data import run_cleaning_pipeline
from feature_engineering.build_features import run_feature_engineering_pipeline

In [2]:
# Configuration du logger pour voir les infos dans le notebook
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")
logger = logging.getLogger("notebook")

<a id="section-1"></a>
# Section 1:Chargement des data

In [ ]:
cfg = load_config()
create_directories(cfg) # créer les dossiers si absent

# A. Chargement Raw
logger.info("--- 1. LOADING ---")
df_raw = load_data_raw(cfg)

# B. Nettoyage 
logger.info("--- 2. CLEANING ---")
df_cleaned = run_cleaning_pipeline(df_raw, cfg)

# C. Feature Engineering 
logger.info("--- 3. FEATURE ENGINEERING ---")

df_final = run_feature_engineering_pipeline(df_cleaned, cfg)

# 5. Vérification
print(f"Données prêtes pour le modèle : {df_final.shape}")


2026-01-10 23:52:22,845 - Configuration 'config' chargée (project_root=C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2)
2026-01-10 23:52:22,845 - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\raw
2026-01-10 23:52:22,852 - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\interim
2026-01-10 23:52:22,852 - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\processed
2026-01-10 23:52:22,854 - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\figures
2026-01-10 23:52:22,858 - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\reports
2026-01-10 23:52:22,859 - --- 1. LOADING ---
2026-01-10 23:52:22,893 - DataFrame chargé : 3376 lignes, 46 colonnes
2026-01-10 23:52:22,911 - ✔️ 2016_Building_Energy_Benchmarking.csv : Identique à la version précédente.
2026-01-10 23:52:22,912 - --- 2. CLEANING ---
2026-01-10 2

   [Audit] section_0 : -1752 lignes exportées vers section_0_removed.csv
   [Audit] section_1 : -119 lignes exportées vers section_1_removed.csv
   [Audit] section_2 : -33 lignes exportées vers section_2_removed.csv


2026-01-10 23:52:23,727 - ✓ Données sauvegardées dans : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\interim\data_cleaned.csv
2026-01-10 23:52:23,732 - --- 3. FEATURE ENGINEERING ---
2026-01-10 23:52:23,732 - --- Démarrage : Feature Engineering ---
2026-01-10 23:52:23,840 - ✓ Feature Engineering terminé. Shape: (1407, 61)


   [Audit] section_3 : -65 lignes exportées vers section_3_removed.csv
✓ Feature engineering sauvegarde dans : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\processed\model_input.csv
Données prêtes pour le modèle : (1407, 61)


<a id="section-2"></a>
# Section 2 : Chargement et inspection structurelle

Cette section se concentre sur le chargement initial des données à partir du fichier CSV et une inspection structurelle de base. L'objectif est d'obtenir une première vue d'ensemble du dataset, de vérifier son intégrité et d'identifier tout problème évident au niveau du format ou de la structure.

## Importation du CSV

In [3]:
# Chargement des données brutes
df_raw = load_data_raw(cfg)

# Validation dimensions
n_rows, n_cols = df_raw.shape
logger.info(f"Dataset chargé ({n_rows} lignes, {n_cols} colonnes)")



2025-12-31 12:36:31,899 - data.load_data - INFO - DataFrame chargé : 3376 lignes, 46 colonnes
2025-12-31 12:36:31,955 - data.load_data - INFO -  Intégrité des données validée (Aucune modification détectée).
2025-12-31 12:36:31,957 - __main__ - INFO - Dataset chargé (3376 lignes, 46 colonnes)


---
Le chargement nous donne un DataFrame avec 3376 lignes et 46 colonnes.

## Affichage pour première impression visuelle
---

In [4]:
# Forcer pandas à afficher toutes les colonnes
pd.set_option("display.max_columns", None)

# Forcer pandas à afficher toutes les lignes (si besoin)
pd.set_option("display.max_rows", None)

# Forcer pandas à afficher toute la largeur (évite les "...")
pd.set_option("display.width", None)

